A couple of days ago, I stumbled upon the network visualizations of travel logs made by Pieter Levels on his NomadList website. NomadList is a website that builds the infrastructure to help digital nomads live anywhere in the world and connect with like-minded people. So I asked Pieter on Twitter if he could share the underlying data for travel logs, and lo and behold, the very next day, he made the aggregated travel logs data publicly available. The only fitting next step for me was to take that data and plug it into Neo4j and play around with it. This blog post will present some of my findings and also show you how to use the new Link Prediction Pipeline to predict new connections in a graph.

If you want to follow along with examples in this post, I recommend using a Blank project in Neo4j Sandbox. It is a free cloud instance of Neo4j database that comes pre-installed with both APOC and Graph Data Science plugins.

In [1]:
# Define Neo4j connections
import pandas as pd
from neo4j import GraphDatabase
host = 'bolt://3.86.246.160:7687'
user = 'neo4j'
password = 'pot-davit-screwdrivers'
driver = GraphDatabase.driver(host,auth=(user, password))

def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

The graph model consists of cities, which are represented as nodes, and travel logs described as relationships.The travel logs are already aggregated in the underlying data, so we only know how many travels were made from one city to another. Unfortunately, there is no time component in the data, so we can't compare pre and post-pandemic travels or how long a person stayed in that city. All we know is that all travels happened somewhere between 2014 and 2021, and the data is available for top 50 cities per continent. On a more technical level, we are dealing with a directed weighted graph.
# Graph import
If you want to follow along, open your Neo4j Sandbox instance and copy the Cypher queries as we go along. I have also prepared all the code in a form of Jupyter Notebook if you want to use that.
First, we will define a unique property constraint on City nodes. The unique property constraint ensures that property values are unique for all nodes of a specific label, but also automatically adds an index to that property for faster imports and queries.

In [2]:
run_query("""
CREATE CONSTRAINT ON (c:City) ASSERT c.id IS UNIQUE;
""")

""


Next, we will import the travel logs from the NomadList website. They are conveniently available in JSON format on the website. We can use the apoc.load.json procedure to retrieve the data from the page. We will also use apoc.periodic.iterate procedure for batching purposes. Read more about how to batch transactions in the documentation.

In [3]:
run_query("""
CALL apoc.periodic.iterate('
  CALL apoc.load.json("https://nomadlist.com/graph.json")
  YIELD value
  WITH value, [x in keys(value) WHERE x <> "README" | x] AS keys
  UNWIND keys AS source_city
  WITH source_city, value
  RETURN source_city, value
','
  MERGE (s:City{name:source_city})
  WITH value[source_city] as destinations, s
  WHERE destinations <> []
  WITH destinations, keys(destinations) as destination_cities, s
  UNWIND destination_cities AS destination_city
  MERGE (t:City{name:destination_city})
  MERGE (s)-[r:TRAVEL_TO]->(t)
  SET r.weight = destinations[destination_city]', 
  {batchSize:10})
""")

,batches,total,timeTaken,committedOperations,failedOperations,failedBatches,retries,errorMessages,batch,operations,wasTerminated,failedParams,updateStatistics
0,34,332,32,332,0,0,0,{},"{'total': 34, 'committed': 34, 'failed': 0, 'e...","{'total': 332, 'committed': 332, 'failed': 0, ...",False,{},"{'nodesDeleted': 0, 'labelsAdded': 332, 'relat..."


The distance between cities is not available in the original dataset. You could use APOC spatial procedures to retrieve GPS locations. To avoid spamming the OSM server, I have prepared a CSV file with all the city locations and store it on GitHub. Just like fetching JSON files from the internet, you can also retrieve CSV files from any webpage. Run the following query to import locations of cities.

In [4]:
run_query("""
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/nomad/nomad_cities_location.csv" as row
MATCH (c:City)
WHERE c.name = row.city
SET c.location = point({latitude:toFloat(row.latitude), longitude:toFloat(row.longitude)})
""")

""


Before we jump into analysis, the last thing is to calculate the distance between cities based on their location information.

In [5]:
run_query("""
MATCH (s:City)-[r:TRAVEL_TO]->(t:City)
WITH r,distance(s.location, t.location) / 1000 AS distanceInKm
SET r.distance = distanceInKm
""")

""


# Exploratory graph analysis

Let's first examine how many travel logs are represented in our graph. Remember, the data was already aggregated before we got it, so we need to look at the total sum of the weight property for all travel relationships.

In [6]:
run_query("""
MATCH p=()-[r:TRAVEL_TO]->()
RETURN sum(r.weight) as all_travels
""")

,all_travels
0,27893


In my case, I got a total of 26657 travel logs. You are likely to get a higher number as I believe the data is not static and is updated regularly. Now it is time to examine which are the most popular destinations. For those who like technical terms, we will investigate nodes with the highest weighted in-degree.

In [7]:
run_query("""
MATCH (c:City)<-[r:TRAVEL_TO]-()
RETURN c.name as city, sum(r.weight) as travels
ORDER BY travels DESC
LIMIT 10
""")

,city,travels
0,bangkok-thailand,1576
1,london-united-kingdom,1520
2,new-york-city-ny-united-states,1287
3,chiang-mai-thailand,1114
4,berlin-germany,1019
5,san-francisco-ca-united-states,957
6,paris-france,886
7,singapore-singapore,877
8,barcelona-spain,793
9,amsterdam-netherlands,788


Surprisingly, London is in second place. I would never imagine that, however Bangkok place makes a lot of sense from my perspective. It would be interesting to know if people are only passing through London to get to their final destination or are they actually staying there longer. We lack that information, so we can only hypothesize. All in all, seems that Thailand and Western Europe are popular destinations for digital nomads. For good measure, also Singapore, New York and San Francisco are included.
I am also interested in how far do people like to travel on average. We added the distance property to relationships so that we can examine their distribution roughly. I've prepared a Cypher query that creates four bins and calculates the ratios of travels within that distance divided by all the travels.

In [8]:
run_query("""
MATCH ()-[r:TRAVEL_TO]->()
RETURN sum(CASE WHEN r.distance < 500 THEN r.weight END) 
          / toFloat(sum(r.weight)) AS within_500,
sum(CASE WHEN 500 < r.distance < 1000 THEN r.weight END) 
          / toFloat(sum(r.weight)) AS within_1000,
sum(CASE WHEN 1000 < r.distance < 2000 THEN r.weight END) 
          / toFloat(sum(r.weight)) AS within_2000,
sum(CASE WHEN 2000 < r.distance  THEN r.weight END) 
          / toFloat(sum(r.weight)) AS rest
""")

,within_500,within_1000,within_2000,rest
0,0.265658,0.21371,0.215144,0.269924


Almost 50% of all logged travels are less than 1000km. As a European, I imagine that trips up to 1000km are one night-train away, and as far I have heard for Americans, that's almost like a commute to work. Even a 2000km trip is, on average, only a two-hour flight. For the rest 30% of travels, I imagine they are most likely cross-continental trips. My hypothesis is that digital nomads like to travel across continents, and then explore that continent by moving from city to city. Of course, this is only my hypothesis, and I could be wrong.
# Graph Data Science
Now we will take advantage of graph algorithms available in the Neo4j Graph Data Science library for our further analysis. We will begin by projecting in-memory graph that will be used as basis for executing graph algorithms. If you want to learn more about the Graph Data Science library and how it works, I would suggest the free Introduction to Graph Algorithms course.

In [9]:
run_query("""
CALL gds.graph.create('nomad', 'City', 'TRAVEL_TO', {relationshipProperties:'weight'});
""")

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,createMillis
0,"{'City': {'properties': {}, 'label': 'City'}}","{'TRAVEL_TO': {'orientation': 'NATURAL', 'aggr...",nomad,332,2407,320


Almost any graph analysis starts with Weakly-Connected component algorithm. It is used to find disconnected parts or islands of nodes within the graph. To fetch the high-level stats of the WCC results, you can execute the following query:

In [10]:
run_query("""
CALL gds.wcc.stats('nomad')
YIELD componentCount, componentDistribution;
""")

,componentCount,componentDistribution
0,32,"{'p99': 301, 'min': 1, 'max': 301, 'mean': 10...."


The WCC algorithm found 38 disconnected components, with the largest having 301 members. Judging by the percentile values, it seems that there is a single super component with 301 members, and then there are 31 components with only a single member. To inspect the cities that don't have any connection with the rest of the world, we can use the following Cypher query:

In [11]:
run_query("""
MATCH (c:City)
WHERE NOT (c)--()
RETURN c.name as city
LIMIT 10;
""")

,city
0,boumerdas-algeria
1,lubumbashi-dr-congo
2,onitsha-nigeria
3,ouagadougou-burkina-faso
4,hama-syria
5,bryan-tx-united-states
6,zaria-nigeria
7,kananga-dr-congo
8,oshawa-canada
9,port-said-egypt


While these cities are mentioned in the NomadList dataset, they don't have any connection with the outer world.
# Betweenness centrality
Next, we will take a look at the Betweenness centrality. The Betweenness centrality is used to identify bridges or connections between clusters of nodes. It is calculated by finding the shortest paths between all pairs of nodes and then looking at the count of when a node appears on those shortest paths. You can execute the betweenness centrality on the nomadlist projected in-memory graph as follows:

In [12]:
run_query("""
CALL gds.betweenness.stream('nomad')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS city, score
ORDER BY score DESC LIMIT 10
""")

,city,score
0,dubai-united-arab-emirates,8284.948041
1,london-united-kingdom,6019.700963
2,bangkok-thailand,5475.905508
3,manama-bahrain,5206.527669
4,istanbul-turkey,4538.393930
5,san-francisco-ca-united-states,4237.947584
6,mexico-city-mexico,3909.739982
7,cape-town-south-africa,3705.800335
8,medellin-colombia,3584.430216
9,vancouver-canada,3420.495107


One important disclaimer is that, in this example, the betweenness centrality doesn't take the relationship weights into account. If you were to take a trip around the world and follow the typical digital nomad travel patterns, you would likely end up at these cities at some point in time. For example, if you travel between Europe and Asia, you have good chances of ending up in Dubai sooner or later. I already mentioned before that I would be interested in learning if nomads stay longer in London or visit it only to fly out across continents. Similarly, you could interpret these results as having New York and Mexico be entry or exit points to North America.
# Network visualizations
Now, we will create a lovely feature image for this blog post. I am only partially kidding, but with smaller datasets like this one, it makes sense to develop nice network visualizations as an excellent visualization can be worth more than a thousand words.
We will use the PageRank score to determine the size of the nodes. In the context of PageRank, each relationship can be interpreted as a vote of confidence. In our example with the NomadList travel logs, the most popular nomad destinations should have the highest score. This is because the PageRank algorithm considers both the number of incoming connections as well as where those connections are coming from.

In [13]:
run_query("""
CALL gds.pageRank.write('nomad', {relationshipWeightProperty:'weight', writeProperty:'pagerank'});
""")

,writeMillis,nodePropertiesWritten,ranIterations,didConverge,centralityDistribution,postProcessingMillis,createMillis,computeMillis,configuration
0,697,332,20,False,"{'p99': 10.039488792419434, 'min': 0.149999618...",1289,4,542,"{'maxIterations': 20, 'writeConcurrency': 4, '..."


To determine the color of the nodes in our visualization, we will use the Louvain Modularity algorithm. The Louvain Modularity algorithm is a community detection or clustering algorithm that groups together nodes that are densely connected.

In [14]:
run_query("""
CALL gds.louvain.write('nomad', {relationshipWeightProperty:'weight', writeProperty:'louvain'});
""")

,writeMillis,nodePropertiesWritten,modularity,modularities,ranLevels,communityCount,communityDistribution,postProcessingMillis,createMillis,computeMillis,configuration
0,292,332,0.560238,[0.560237539768752],1,54,"{'p99': 62, 'min': 1, 'max': 65, 'mean': 6.148...",4,0,3766,"{'maxIterations': 10, 'writeConcurrency': 4, '..."


If you are using Neo4j Sandbox or Desktop, you can open the Neo4j Bloom and recreate the following visualization. Check the documentation if you want to learn more about rule-based network visualization in Neo4j Bloom.

# Link prediction pipeline
Link Prediction Pipeline was added in the 1.7 version of the Neo4j Graph Data Science library. Link prediction is a technique that does exactly what you would imagine. It is used to predict new connections in the network that do not exists yet. You can think of it as a binary classification model that indicates if a new link is likely to exist between pairs of nodes. There are several applications for link prediction ranging from recommender systems to drug repurposing. To start, you first have to define a new link prediction pipeline.

In [24]:
run_query("""
CALL gds.alpha.ml.pipeline.linkPrediction.create('lp-pipeline');
""")

,name,nodePropertySteps,featureSteps,splitConfig,parameterSpace
0,lp-pipeline,[],[],"{'negativeSamplingRatio': 1.0, 'trainFraction'...","[{'useBiasFeature': True, 'maxEpochs': 100, 'm..."


As a part of the link prediction pipeline, you can add intermediate steps where other graph algorithms are executed. You want to run graph algorithms which output will be used as link prediction classifier features. In this example, we will be using the FastRP node embeddings as input features for the link prediction model. You can read more about the FastRP algorithm in this excellent article by my friend Clair Sullivan. 
https://towardsdatascience.com/behind-the-scenes-on-the-fast-random-projection-algorithm-for-generating-graph-embeddings-efb1db0895

In [25]:
run_query("""
CALL gds.alpha.ml.pipeline.linkPrediction.addNodeProperty(
  'lp-pipeline', 
  'fastRPExtended', {
    mutateProperty: 'embedding',
    featureProperties:['pagerank'],
    embeddingDimension: 128,
    randomSeed: 42
});
""")

,name,nodePropertySteps,featureSteps,splitConfig,parameterSpace
0,lp-pipeline,"[{'name': 'gds.beta.fastRPExtended.mutate', 'c...",[],"{'negativeSamplingRatio': 1.0, 'trainFraction'...","[{'useBiasFeature': True, 'maxEpochs': 100, 'm..."


Here, we have defined that the link prediction pipeline should start by executing the FastRP algorithm, which will output an embedding for all the nodes in the network. You can also fine-tune algorithm hyper-parameters in the configuration of this step. Next, we need to define how to combine a link's source and target node features to produce an input to the link prediction model. Remember, the link prediction model in Neo4j GDS is a binary classification model that uses logistic regression under the hood. The output is either a 1 or 0 if a connection exists in the network or not, and the input features are combined by considering both source and target node features. At the moment, the pipeline features three different ways of combining node features:
* L2 
* Hadamard
* Cosine

You can inspect the documentation for more information about combining node features as an input to the link prediction model. In this example, we will use the L2 combiner to combine the FastRP node embeddings of pairs of nodes.

In [26]:
run_query("""
CALL gds.alpha.ml.pipeline.linkPrediction.addFeature('lp-pipeline', 'cosine', {
  nodeProperties: ['embedding']
}) YIELD featureSteps;
""")

,featureSteps
0,"[{'name': 'COSINE', 'config': {'nodeProperties..."


Now it is time to define the train-test split. We can define the test-train split ratio. The pipeline uses k-fold crossvalidation training technique. A nice thing about the train-test split is that we don't have to develop a manual data split.

In [27]:
run_query("""
CALL gds.alpha.ml.pipeline.linkPrediction.configureSplit(
 'lp-pipeline', {  
   testFraction: 0.3,
   trainFraction: 0.6,
   validationFolds: 7})
YIELD splitConfig;
""")

,splitConfig
0,"{'negativeSamplingRatio': 1.0, 'trainFraction'..."


The last configuration we have to set before we can train the model are link prediction model hyperparameters. You can provide many variations of model hyperparameters. In this example, we will provide three different combinations of hyperparameters and the model will automatically pick the combination that provides the best results during training.

In [28]:
run_query("""
CALL gds.alpha.ml.pipeline.linkPrediction.configureParams(
  'lp-pipeline',  
    [{tolerance: 0.001, maxEpochs: 500},    
     {penalty:0.001, tolerance: 0.01,  maxEpochs: 500},      
     {penalty:0.01, tolerance: 0.01, maxEpochs: 500}])
YIELD parameterSpace;
""")

,parameterSpace
0,"[{'useBiasFeature': True, 'maxEpochs': 500, 'm..."


Again, we need to project the in-memory graph before we start. The subtle difference from before is that here we are projecting the relationships as undirected. At the moment, the link prediction pipeline supports predicting only undirected relationships.

In [29]:
run_query("""
CALL gds.graph.create('lp-graph', 
  'City', 
  {TRAVEL_TO:{orientation:'UNDIRECTED'}}, 
  {nodeProperties:['pagerank']});
""")

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,createMillis
0,{'City': {'properties': {'pagerank': {'propert...,"{'TRAVEL_TO': {'orientation': 'UNDIRECTED', 'a...",lp-graph,332,4814,104


And finally, we get to train our link prediction model.

In [30]:
run_query("""
CALL gds.alpha.ml.pipeline.linkPrediction.train('lp-graph', 
  {pipeline: 'lp-pipeline',
   modelName: 'lp-model',
   randomSeed: 42})
YIELD modelInfo
RETURN  modelInfo.bestParameters AS winningModel,  modelInfo.metrics.AUCPR.outerTrain AS trainGraphScore,  modelInfo.metrics.AUCPR.test AS testGraphScore;
""")

,winningModel,trainGraphScore,testGraphScore
0,"{'useBiasFeature': True, 'maxEpochs': 500, 'mi...",0.777271,0.831283


You can use the mutate mode to produce classification results and store the predicted relationships back to the projected graph.

In [31]:
run_query("""
CALL gds.alpha.ml.pipeline.linkPrediction.predict.mutate('lp-graph', 
  {modelName: 'lp-model',  
   mutateRelationshipType: 'TRAVEL_PREDICTED',
   topN: 20,
   threshold: 0.45})
YIELD relationshipsWritten;
""")

,relationshipsWritten
0,40


As the final step, you can stream and inspect results with the following Cypher query.

In [32]:
run_query("""
CALL gds.graph.streamRelationshipProperty('lp-graph', 
  'probability', 
  ['TRAVEL_PREDICTED'])
YIELD  sourceNodeId, targetNodeId, propertyValue
WHERE sourceNodeId < targetNodeId
WITH  gds.util.asNode(sourceNodeId).name as city1, gds.util.asNode(targetNodeId).name as city2, propertyValue AS probability
ORDER BY probability DESC
RETURN city1, city2
LIMIT 10;
""")

,city1,city2
0,conakry-guinea,abbotsford-canada
1,perth-australia,cairns-australia
2,madeira-portugal,ericeira-portugal
3,tangier-morocco,agadir-morocco
4,hagatna-guam,busan-south-korea
5,perth-australia,byron-bay-australia
6,yangon-myanmar,luang-prabang-laos
7,puerto-vallarta-mexico,puerto-escondido-mexico
8,penang-malaysia,yangon-myanmar
9,las-vegas-nv-united-states,tucson-az-united-states


Our link prediction mostly predicts new local connections. For example, it predicts new travels between Madeira and Ericeira in Portual or Tangier and Agadir in Morocco. Similarly, it predicts new connections between neighhbouring countries like Cambodia, Vietnam, and Thailand. The only prediction that seems a bit off is connecting Conakry in Guinea to Abbotsford in Canada.